#Requirements

In [ ]:
!pip install -r https://raw.githubusercontent.com/ProbablyItsSpirit/Music_Recommender_System/refs/heads/main/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 428.3 kB/s eta 0:00:00
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357279 sha256=eae28c09a8d2dfc0835a8995cb72c76d6679833045ccc6b606b86f303eaecf6d
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from tabulate import tabulate
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from wordcloud import WordCloud
pd.options.mode.chained_assignment = None
from sklearn.preprocessing import MultiLabelBinarizer
import category_encoders as ce
import seaborn as sb
import numpy as np
from scipy.interpolate import CubicSpline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
g= 'https://raw.githubusercontent.com/ProbablyItsSpirit/MRS_ML/refs/heads/main/Dataset/tracks_new.csv'
df = pd.read_csv(g)
o= 'https://raw.githubusercontent.com/ProbablyItsSpirit/MRS_ML/refs/heads/main/Dataset/ratings.csv'
df2 = pd.read_csv(o)

In [ ]:
merged_df = pd.merge(df, df2, on='song_id', how='inner', suffixes=('_df', '_df2'))

df2 = pd.merge(df2, df[['song_id', 'release_date']], on='song_id', how='left')


In [ ]:
df.head(1)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,danceability_norm,energy_norm,tempo_norm,loudness_norm,acousticness_norm,instrumentalness_norm,speechiness_norm,liveness_norm,valence_norm,enriched_tags
0,1,peaches (feat. daniel caesar & giveon),100,198082,1,"Justin Bieber, Daniel Caesar, Giveon",7369,2021-03-19,0.677,0.696,...,0.685223,0.695994,0.369928,0.86161,0.322289,0.0,0.126059,0.42,0.464,peaches (feat. daniel caesar & giveon) ['Justi...


In [ ]:
def enrich_tags_with_features(df):
    """
    Enrich the tags column by adding normalized numerical features (danceability, energy, etc.).
    """
    # Normalize numerical features to avoid dominance by large values
    numerical_columns = ['danceability', 'energy', 'tempo', 'loudness', 'acousticness',
                         'instrumentalness', 'speechiness', 'liveness', 'valence']

    # Normalize and add as string representation of normalized features to the tags
    for col in numerical_columns:
        max_value = df[col].max()
        min_value = df[col].min()
        df[f'{col}_norm'] = (df[col] - min_value) / (max_value - min_value)  # Min-Max Normalization

    # Combine the original tags with the normalized features
    df['enriched_tags'] = df['tags'] + " " + df[numerical_columns].apply(lambda row: ' '.join([f"{col}:{row[col]:.2f}" for col in numerical_columns]), axis=1)

    return df

#Recommender

##Popularity Based Recommender System

In [ ]:
# Popularity Based Recommendation
trend = df.groupby('name', as_index=False)['popularity'].mean()
trend = trend.sort_values(by='popularity', ascending=False).head(10)

In [ ]:
trend = df.groupby('name', as_index=False)['popularity'].mean()
top_trending = trend.sort_values(by='popularity', ascending=False)

print("Popularity Based Recommendation System: (Trending Songs)")

trending_details = df.merge(top_trending, on='name', how='inner')

trending_details = trending_details[['id', 'name', 'artists', 'release_date', 'explicit', 'popularity_y']]
trending_details = trending_details.rename(columns={'popularity_y': 'popularity'})

# Remove duplicates to ensure unique songs and sort again if needed
trending_details = trending_details.drop_duplicates(subset=['id']).sort_values(by='popularity', ascending=False)

trending_details

Popularity Based Recommendation System: (Trending Songs)


,id,name,artists,release_date,explicit,popularity
0,1,peaches (feat. daniel caesar & giveon),"Justin Bieber, Daniel Caesar, Giveon",2021-03-19,1,100.0
1,2,drivers license,Olivia Rodrigo,2021-01-08,1,99.0
2,3,astronaut in the ocean,Masked Wolf,2021-01-06,0,98.0
3,4,telepatía,Kali Uchis,2020-12-04,0,97.0
4,5,save your tears,The Weeknd,2020-03-20,1,97.0
...,...,...,...,...,...,...
69767,69768,vasame,Arisa,2017-12-29,0,49.0
69765,69766,bedingungslos,Sarah Connor,2015-05-22,0,49.0
69764,69765,küçük bir aşk masalı,Özdemir Erdoğan,1987,0,49.0
69763,69764,the angels voice,Diane Arkenstone,2001-03-13,0,49.0


##Content Based Recommendation System

### Using Artist Name and Tags

In [ ]:
def content_based_recommendation(df, sname, top_n=10):
    # Song Check
    if sname not in df['name'].values:
        print(f"Sorry, cannot find any song named '{sname}', perhaps try something else?")
        return pd.DataFrame()

    df = enrich_tags_with_features(df)
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix_content = tfidf_vectorizer.fit_transform(df['enriched_tags'])

    knn = NearestNeighbors(n_neighbors=top_n+1, metric='cosine', n_jobs=-1)
    knn.fit(tfidf_matrix_content)

    song_index = df[df['name'] == sname].index[0]

    distances, indices = knn.kneighbors(tfidf_matrix_content[song_index], n_neighbors=top_n+1)

    recommended_song_indices = indices[0][1:]

    recommended_songs_details = df.iloc[recommended_song_indices][['name', 'artists', 'release_date']]

#OtherSongs
    artist_name = df.iloc[song_index]['artists']
    other_songs_by_artist = df[(df['artists'] == artist_name) & (df['name'] != sname)]
    other_songs_by_artist = other_songs_by_artist.head(5)

    print(f"More songs like '{sname}': ")
    print(recommended_songs_details[['name', 'artists', 'release_date']].to_string(index=False))

    if not other_songs_by_artist.empty:
        print("\nOther Songs by the same Artist:")
        print(other_songs_by_artist[['name', 'release_date']].to_string(index=False))
    else:
        print("No other songs by this artist(s) found.")

    return recommended_songs_details


In [ ]:
sname = "zingaat"
recommend_songs = content_based_recommendation(df, sname, top_n=5)
print(tabulate(recommend_songs, headers='keys', tablefmt='fancy_grid'))

More songs like 'zingaat': 
               name                      artists release_date
            zingaat Ajay Gogavale, Atul Gogavale   2016-02-24
    brothers anthem    Ajay-Atul, Vishal Dadlani   2015-07-24
 deva shree ganesha     Ajay-Atul, Ajay Gogavale   2011-12-16
  how you like that                    BLACKPINK   2020-10-02
armed and dangerous                   Juice WRLD   2018-12-10
No other songs by this artist(s) found.
╒═══════╤═════════════════════╤══════════════════════════════╤════════════════╕
│       │ name                │ artists                      │ release_date   │
╞═══════╪═════════════════════╪══════════════════════════════╪════════════════╡
│ 62962 │ zingaat             │ Ajay Gogavale, Atul Gogavale │ 2016-02-24     │
├───────┼─────────────────────┼──────────────────────────────┼────────────────┤
│ 36483 │ brothers anthem     │ Ajay-Atul, Vishal Dadlani    │ 2015-07-24     │
├───────┼─────────────────────┼──────────────────────────────┼────────────────┤


### Using Feature Matrix

In [ ]:
import random

def content_based2_rec(df, sname, top_n=10):
    # Song Check
    if sname not in df['name'].values:
        print(f"Sorry, cannot find any song named '{sname}', perhaps try something else?")
        return pd.DataFrame()

    features = ['danceability_norm', 'energy_norm', 'tempo_norm',
                'loudness_norm', 'acousticness_norm', 'instrumentalness_norm',
                'speechiness_norm', 'liveness_norm', 'valence_norm']

    feature_matrix = df[features].values

    # KNN model
    knn = NearestNeighbors(n_neighbors=top_n+1, metric='cosine', n_jobs=-1)
    knn.fit(feature_matrix)

    song_index = df[df['name'] == sname].index[0]

    distances, indices = knn.kneighbors(feature_matrix[song_index].reshape(1, -1), n_neighbors=top_n+1)

    # Exclude the song itself from recommendations and shuffle
    recommended_song_indices = indices[0][1:]  # Skip the first index (the song itself)
    random.shuffle(recommended_song_indices)  # Shuffle the indices

    # Retrieve recommended songs excluding the current song
    recommended_songs_details = df.iloc[recommended_song_indices][['name', 'artists', 'release_date']]

    # Randomize other songs by the same artist
    artist_name = df.iloc[song_index]['artists']
    other_songs_by_artist = df[(df['artists'] == artist_name) & (df['name'] != sname)]
    other_songs_by_artist = other_songs_by_artist.sample(n=min(5, len(other_songs_by_artist))) if not other_songs_by_artist.empty else pd.DataFrame()

    # Print recommendations
    print(f"More songs like '{sname}': ")
    print(recommended_songs_details[['name', 'artists', 'release_date']].to_string(index=False))

    if not other_songs_by_artist.empty:
        print("\nOther Songs by the same Artist:")
        print(other_songs_by_artist[['name', 'release_date']].to_string(index=False))
    else:
        print("No other songs by this artist(s) found.")

    return recommended_songs_details


In [ ]:
sname = "nancy mulligan"
recommend_songs = content_based2_rec(df, sname, top_n=5)
print(tabulate(recommend_songs, headers='keys', tablefmt='fancy_grid'))

More songs like 'nancy mulligan': 
                       name              artists release_date
           over my shoulder Mike & The Mechanics   1995-03-06
         no podré olvidarte            Grupo Red   1997-10-16
                    báilalo       Garras de Amor   2000-07-25
           im wagen vor mir      Henry Valentino         1994
all my love - 1990 remaster         Led Zeppelin   1979-08-15

Other Songs by the same Artist:
                          name release_date
                    i see fire   2013-12-10
                        u.n.i.   2011-09-09
hearts don't break around here   2017-03-03
                bibia be ye ye   2017-03-03
                   save myself   2017-03-03
╒═══════╤═════════════════════════════╤══════════════════════╤════════════════╕
│       │ name                        │ artists              │ release_date   │
╞═══════╪═════════════════════════════╪══════════════════════╪════════════════╡
│  8087 │ over my shoulder            │ Mike & The Mecha

##Collaborative Filtering

In [ ]:
df.columns

Index(['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artists',
       'id_artists', 'release_date', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature', 'id_artists_encoded',
       'song_id', 'tags', 'danceability_norm', 'energy_norm', 'tempo_norm',
       'loudness_norm', 'acousticness_norm', 'instrumentalness_norm',
       'speechiness_norm', 'liveness_norm', 'valence_norm', 'enriched_tags'],
      dtype='object')

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
#user_item_matrix = df2.pivot_table(index='Uid', columns='song_id', values='rating', aggfunc='mean').fillna(0)
#knn = NearestNeighbors(n_neighbors=k_neighbors, metric='cosine', n_jobs=-1)
#knn.fit(user_item_matrix)

In [ ]:
target_user_id = 4  # You can change this to the ID of the user you're interested in
target_user_index = user_item_matrix.index.get_loc(target_user_id)
knn_model = NearestNeighbors(metric='cosine', n_neighbors=6)
knn_model.fit(user_item_matrix.values)

NearestNeighbors(metric='cosine', n_neighbors=6)

In [ ]:
# prompt: print Ratings by Uid 4

distances, indices = knn_model.kneighbors(user_item_matrix.iloc[target_user_index, :].values.reshape(1, -1))

print(f"Ratings by Uid {target_user_id}:")
for i in range(len(indices.flatten())):
    neighbor_id = user_item_matrix.index[indices.flatten()[i]]
    distance = distances.flatten()[i]
    print(f"Neighbor {neighbor_id}: Distance = {distance}")
    # You can access the ratings of the neighbor using user_item_matrix.loc[neighbor_id]

Ratings by Uid 4:
Neighbor 4: Distance = 1.1102230246251565e-16
Neighbor 20: Distance = 0.9742540484000844
Neighbor 49: Distance = 0.9748119200451058
Neighbor 8: Distance = 0.9748309014384207
Neighbor 23: Distance = 0.9749724912697325
Neighbor 55: Distance = 0.9749851642480956


In [ ]:
distances, indices = knn_model.kneighbors(user_item_matrix.iloc[target_user_index].values.reshape(1, -1), n_neighbors=6)

similar_user_indices = indices.flatten()[distances.argsort()[::-1][1:]]
print(f"Similar user indices: {similar_user_indices}")

Similar user indices: []


In [ ]:
recommend_items = []

for user_index in similar_user_indices:
    rated_by_similar_user = user_item_matrix.iloc[user_index]
    not_rated_by_target_user = (rated_by_similar_user == 0) & (user_item_matrix.iloc[target_user_index] == 0)

    # Add top 10 songs that the target user hasn't rated
    recommend_items.extend(user_item_matrix.columns[not_rated_by_target_user][:10])

# Step 7: Get details for the recommended songs
recommended_items_details = df2[df2['song_id'].isin(recommend_items[:10])][['song_id', 'rating']].drop_duplicates()

# Display the recommended songs
print(recommended_items_details.head(10))

Empty DataFrame
Columns: [song_id, rating]
Index: []


In [ ]:
# prompt: print Ratings by Uid 4

distances, indices = knn_model.kneighbors(user_item_matrix.iloc[target_user_index, :].values.reshape(1, -1))

print(f"Ratings by Uid {target_user_id}:")
for i in range(len(indices.flatten())):
    neighbor_id = user_item_matrix.index[indices.flatten()[i]]
    distance = distances.flatten()[i]
    print(f"Neighbor {neighbor_id}: Distance = {distance}")
    # You can access the ratings of the neighbor using user_item_matrix.loc[neighbor_id]

Ratings by Uid 4:
Neighbor 4: Distance = 1.1102230246251565e-16
Neighbor 20: Distance = 0.9742540484000844
Neighbor 49: Distance = 0.9748119200451058
Neighbor 8: Distance = 0.9748309014384207
Neighbor 23: Distance = 0.9749724912697325
Neighbor 55: Distance = 0.9749851642480956


In [ ]:
# prompt: Print Uid and rating

print(df2[['Uid', 'rating', 'song_id']])

        Uid  rating  song_id
0         1       2    15796
1         1       0      861
2         1       1    54887
3         1       5     6266
4         1       1    37195
...     ...     ...      ...
139995   70       1    39542
139996   70       2    54959
139997   70       0     6758
139998   70       3    39384
139999   70       1    15953

[140000 rows x 3 columns]


In [ ]:
def collaborative_filtering_recommendations(df, target_user_id, top_n=10, k_neighbors=5):
    # Create the user-item matrix (rows = users, columns = songs)
    user_item_matrix = df2.pivot_table(index='Uid', columns='song_id', values='rating', aggfunc='mean').fillna(0)

    # KNN model to find the nearest neighbors (users)
    knn = NearestNeighbors(n_neighbors=k_neighbors, metric='cosine', n_jobs=-1)
    knn.fit(user_item_matrix)

    # Find the nearest neighbors for the target user
    target_user_index = user_item_matrix.index.get_loc(target_user_id)
    distances, indices = knn.kneighbors(user_item_matrix.iloc[target_user_index].values.reshape(1, -1))

    # Get the indices of the similar users (excluding the target user)
    similar_users_indices = indices.flatten()[1:]

    # Generate recommendations based on similar users
    recommended_items = []

    for user_index in similar_users_indices:
        # Get items rated by the similar user but not by the target user
        rated_by_similar_user = user_item_matrix.iloc[user_index]
        not_rated_by_target_user = (rated_by_similar_user > 0) & (user_item_matrix.iloc[target_user_index] == 0)

        # Extract the song IDs of recommended songs
        recommended_items.extend(user_item_matrix.columns[not_rated_by_target_user][:top_n])

    # Get the details of recommended items from your original dataframe
    recommended_items_details = df2[df2['song_id'].isin(recommended_items)][['name', 'popularity', 'artists', 'rating']]

    return recommended_items_details.head(top_n)

# Example usage
target_user_id = 4  # The target user for recommendations
top_n = 5  # Top N songs to recommend
collaborative_filtering_rec = collaborative_filtering_recommendations(df, target_user_id, top_n)
print(f"Top {top_n} recommendations for User {target_user_id}:")
print(collaborative_filtering_rec)

Top 5 recommendations for User 4:
                  name  popularity            artists  rating
5057        heat waves          85      Glass Animals       2
5266      stressed out          86  Twenty One Pilots       0
8811           bichota          91            KAROL G       3
9596           heather          89         Conan Gray       0
11060  blinding lights          96         The Weeknd       4


##Hybrid Recommender

In [ ]:
df2.columns

Index(['Uid', 'song_id', 'rating', 'name_x', 'popularity_x', 'artists_x',
       'name_y', 'popularity_y', 'artists_y', 'name', 'popularity', 'artists',
       'release_date_x', 'release_date_y'],
      dtype='object')

In [ ]:
def hybrid_recommendations(df, target_user_id, sname, top_n=10):
    content_based_rec = content_based_recommendation(df, sname, top_n)
    collaborative_filtering_rec = collaborative_filtering_recommendations(df, target_user_id, top_n)

    # Combine the two recommendation results and drop duplicates based on song name
    hybrid_recommendations = pd.concat([content_based_rec, collaborative_filtering_rec]).drop_duplicates(subset='name')

    # Ensure 'release_date' is in the hybrid recommendations DataFrame
    # If it's present in either content_based_rec or collaborative_filtering_rec:
    if 'release_date' in content_based_rec.columns:
        hybrid_recommendations['release_date'] = hybrid_recommendations['name'].map(content_based_rec.set_index('name')['release_date'])
    elif 'release_date' in collaborative_filtering_rec.columns:  # This is unlikely, but included for completeness
        hybrid_recommendations['release_date'] = hybrid_recommendations['name'].map(collaborative_filtering_rec.set_index('name')['release_date'])
    else:
        # If 'release_date' is not available in either DataFrame, consider adding a message
        print("Warning: 'release_date' is not available in the recommendations.")

    print(f"\nHybrid Recommendations for '{sname}':")
    # If 'release_date' is still not in the DataFrame after the above steps, remove it from the print statement:
    if 'release_date' in hybrid_recommendations.columns:
        print(hybrid_recommendations[['name', 'artists', 'release_date']].to_string(index=False))
    else:
        print(hybrid_recommendations[['name', 'artists']].to_string(index=False))


    return hybrid_recommendations.head(top_n)

In [ ]:
target_user_id = 8
sname = ''
top_n = 5
hybrid_rec = hybrid_recommendations(df, target_user_id, sname, top_n)

More songs like 'faded': 
                              name          artists release_date
                             faded      Alan Walker   2015-12-04
                              fade      Alan Walker   2014-11-19
move your body - alan walker remix Sia, Alan Walker   2016-10-21
                            ransom        Lil Tecca   2019-08-30
                         the other             Lauv   2018-05-31

Other Songs by the same Artist:
            name release_date
     the spectre   2017-09-15
sing me to sleep   2018-12-14
         spectre   2015-01-06
           alone   2016-12-02
            fade   2014-11-19

Hybrid Recommendations for 'faded':
                              name           artists release_date
                             faded       Alan Walker   2015-12-04
                              fade       Alan Walker   2014-11-19
move your body - alan walker remix  Sia, Alan Walker   2016-10-21
                            ransom         Lil Tecca   2019-08-30
    